In [5]:
%load_ext autoreload
%autoreload 2
from glob import glob
import gc
import os
import sys
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_numeric_features, get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from func.time_utils import date_add_days
from func.ml_utils import save_feature, get_cnt_feature, get_dummie_feature, get_label_feature
from func.parallel_utils import get_parallel_arg_list
from kaggle_utils import move_feature
from joblib import delayed, Parallel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET, 'ProductCD']

train_paths = glob('../feature/eda_base/*_train.gz')
test_paths = glob('../feature/eda_base/*_test.gz')
train_paths += glob('../feature/tmp/*_train.gz')
test_paths  += glob('../feature/tmp/*_test.gz')

train_paths = [path for path in train_paths 
               if (path.count(COLUMN_DT) 
               or path.count(COLUMN_ID)
               or path.count(COLUMN_TARGET)
               or path.count('V')
               or path.count('TransactionAmt')
               or path.count('Product')
               )
               and not path.count('fill')
               and not path.count('bin')
               and not path.count('129')
              ]
test_paths = [path for path in test_paths 
               if (path.count(COLUMN_DT) 
               or path.count(COLUMN_ID)
               or path.count(COLUMN_TARGET)
               or path.count('V')
               or path.count('TransactionAmt')
               or path.count('Product')
               )
               and not path.count('fill')
               and not path.count('bin')
               and not path.count('129')
              ]

df_train = parallel_load_data(train_paths)
df_test = parallel_load_data(test_paths)
data = pd.concat([df_train, df_test], axis=0, ignore_index=True)
train_length = df_train.shape[0]

In [3]:
cols_V2_11 = [f'V{no}' for no in np.arange(2, 12, 1)]
cols_V12_34 = [f'V{no}' for no in np.arange(12, 35, 1)]
cols_V35_52 = [f'V{no}' for no in np.arange(35, 53, 1)]
cols_V53_74 = [f'V{no}' for no in np.arange(53, 75, 1)]
cols_V75_94 = [f'V{no}' for no in np.arange(75, 95, 1)]
cols_V95_137 = [f'V{no}' for no in np.arange(95, 138, 1)]
cols_V138_166 = [f'V{no}' for no in np.arange(138, 167, 1)]
cols_V167_216 = [f'V{no}' for no in np.arange(167, 217, 1)]
cols_V217_278 = [f'V{no}' for no in np.arange(217, 279, 1)]
cols_V279_321 = [f'V{no}' for no in np.arange(279, 322, 1)]
cols_V322_339 = [f'V{no}' for no in np.arange(322, 340, 1)]

In [14]:
#========================================================================
# V2~V11
#========================================================================
# display(df_train[cols_V2_11])
# for col in cols_V2_11:
#     display(df_train[col].value_counts(dropna=False).head())
#     display(df_train.groupby(col)[COLUMN_TARGET].mean().head())

data['V2_11_mean'] = data[cols_V2_11].mean(axis=1)
data['V2_11_sum'] = data[cols_V2_11].sum(axis=1)
data['V2_11_std'] = data[cols_V2_11].std(axis=1)
data['V2_11_max'] = data[cols_V2_11].max(axis=1)
data['V2_11_pro'] = data[cols_V2_11].product(axis=1)

data['V3_4_5_9_11_mean'] = data[['V3', 'V4', 'V5', 'V9', 'V11']].mean(axis=1)
data['V3_4_5_9_11_sum']  = data[['V3', 'V4', 'V5', 'V9', 'V11']].sum(axis=1)
data['V3_4_5_9_11_std']  = data[['V3', 'V4', 'V5', 'V9', 'V11']].std(axis=1)
data['V3_4_5_9_11_max']  = data[['V3', 'V4', 'V5', 'V9', 'V11']].max(axis=1)
data['V3_4_5_9_11_pro']  = data[['V3', 'V4', 'V5', 'V9', 'V11']].product(axis=1)

cols_save = ['V2_11_mean'] + ['V2_11_sum']  + ['V2_11_std']  + ['V2_11_max']  + ['V2_11_pro'] +\
['V3_4_5_9_11_mean'] + ['V3_4_5_9_11_sum']  + ['V3_4_5_9_11_std']  + ['V3_4_5_9_11_max']  + ['V3_4_5_9_11_pro']

train = data.iloc[:train_length][cols_save]
test  = data.iloc[train_length:][cols_save]

prefix = '526'
dir_save = 'create'

save_feature(train[cols_save], prefix, dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
save_feature(test[cols_save],  prefix, dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)

(590540,) | V2_11_mean
(590540,) | V2_11_sum
(590540,) | V2_11_std
(590540,) | V2_11_max
(590540,) | V2_11_pro
(590540,) | V3_4_5_9_11_mean
(590540,) | V3_4_5_9_11_sum
(590540,) | V3_4_5_9_11_std
(590540,) | V3_4_5_9_11_max
(590540,) | V3_4_5_9_11_pro
(506691,) | V2_11_mean
(506691,) | V2_11_sum
(506691,) | V2_11_std
(506691,) | V2_11_max
(506691,) | V2_11_pro
(506691,) | V3_4_5_9_11_mean
(506691,) | V3_4_5_9_11_sum
(506691,) | V3_4_5_9_11_std
(506691,) | V3_4_5_9_11_max
(506691,) | V3_4_5_9_11_pro


In [16]:
#========================================================================
# V12~V34
#========================================================================
# display(df_train[cols_V2_11])
# for col in cols_V2_11:
#     display(df_train[col].value_counts(dropna=False).head())
#     display(df_train.groupby(col)[COLUMN_TARGET].mean().head())

if 'V14' in cols_V12_34:
    cols_V12_34.remove('V14')
if 'V27' in cols_V12_34:
    cols_V12_34.remove('V27')
if 'V28' in cols_V12_34:
    cols_V12_34.remove('V28')
if 'V29' in cols_V12_34:
    cols_V12_34.remove('V29')
if 'V30' in cols_V12_34:
    cols_V12_34.remove('V30')

data['V12_34_mean'] = data[cols_V12_34].mean(axis=1)
data['V12_34_sum']  = data[cols_V12_34].sum(axis=1)
data['V12_34_std']  = data[cols_V12_34].std(axis=1)
data['V12_34_max']  = data[cols_V12_34].max(axis=1)
data['V12_34_pro']  = data[cols_V12_34].product(axis=1)

cols_save =  ['V12_34_mean'] + ['V12_34_sum'] + ['V12_34_std'] + ['V12_34_max'] + ['V12_34_pro']  

train = data.iloc[:train_length][cols_save]
test  = data.iloc[train_length:][cols_save]

prefix = '526'
dir_save = 'create'

save_feature(train[cols_save], prefix, dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
save_feature(test[cols_save],  prefix, dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)

(590540,) | V12_34_mean
(590540,) | V12_34_sum
(590540,) | V12_34_std
(590540,) | V12_34_max
(590540,) | V12_34_pro
(506691,) | V12_34_mean
(506691,) | V12_34_sum
(506691,) | V12_34_std
(506691,) | V12_34_max
(506691,) | V12_34_pro


In [20]:
#========================================================================
# V35~52
#========================================================================
# display(df_train[cols_V2_11])
# for col in cols_V2_11:
#     display(df_train[col].value_counts(dropna=False).head())
#     display(df_train.groupby(col)[COLUMN_TARGET].mean().head())

if 'V41' in cols_V35_52:
    cols_V35_52.remove('V41')
    
cols_other = [f'V{no}' for no in [35, 48, 49, 50]]

print(len(cols_V35_52))
cols_V35_52 = list(set(cols_V35_52) - set(cols_other))
print(len(cols_V35_52))

data['V35_52_mean'] = data[cols_V35_52].mean(axis=1)
data['V35_52_sum']  = data[cols_V35_52].sum(axis=1)
data['V35_52_std']  = data[cols_V35_52].std(axis=1)
data['V35_52_max']  = data[cols_V35_52].max(axis=1)
data['V35_52_pro']  = data[cols_V35_52].product(axis=1)

data['V35_52_other_sum']  = data[cols_other].sum(axis=1)
data['V35_52_other_max']  = data[cols_other].max(axis=1)

cols_save =  ['V35_52_mean'] + ['V35_52_sum'] + ['V35_52_std'] + ['V35_52_max'] + ['V35_52_pro']  \
+ ['V35_52_other_mean'] + ['V35_52_other_sum'] + ['V35_52_other_std'] + ['V35_52_other_max'] + ['V35_52_other_pro']

train = data.iloc[:train_length][cols_save]
test  = data.iloc[train_length:][cols_save]

prefix = '526'
dir_save = 'create'

save_feature(train[cols_save], prefix, dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
save_feature(test[cols_save],  prefix, dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)

(590540,) | V35_52_mean
(590540,) | V35_52_sum
(590540,) | V35_52_std
(590540,) | V35_52_max
(590540,) | V35_52_pro
(590540,) | V35_52_other_mean
(590540,) | V35_52_other_sum
(590540,) | V35_52_other_std
(590540,) | V35_52_other_max
(590540,) | V35_52_other_pro
(506691,) | V35_52_mean
(506691,) | V35_52_sum
(506691,) | V35_52_std
(506691,) | V35_52_max
(506691,) | V35_52_pro
(506691,) | V35_52_other_mean
(506691,) | V35_52_other_sum
(506691,) | V35_52_other_std
(506691,) | V35_52_other_max
(506691,) | V35_52_other_pro


In [23]:
#========================================================================
# V53_74
#========================================================================
# display(df_train[cols_V2_11])
# for col in cols_V2_11:
#     display(df_train[col].value_counts(dropna=False).head())
#     display(df_train.groupby(col)[COLUMN_TARGET].mean().head())

if 'V65' in cols_V53_74:
    cols_V53_74.remove('V65')
    
cols_other = [f'V{no}' for no in [68, 69, 70]]

print(len(cols_V53_74))
cols_V53_74 = list(set(cols_V53_74) - set(cols_other))
print(len(cols_V53_74))

data['V53_74_mean'] = data[cols_V53_74].mean(axis=1)
data['V53_74_sum']  = data[cols_V53_74].sum(axis=1)
data['V53_74_std']  = data[cols_V53_74].std(axis=1)
data['V53_74_max']  = data[cols_V53_74].max(axis=1)
data['V53_74_pro']  = data[cols_V53_74].product(axis=1)

data['V53_74_other_sum']  = data[cols_other].sum(axis=1)
data['V53_74_other_max']  = data[cols_other].max(axis=1)

cols_save =  ['V53_74_mean'] + ['V53_74_sum'] + ['V53_74_std'] + ['V53_74_max'] + ['V53_74_pro']  \
+ ['V53_74_other_mean'] + ['V53_74_other_sum'] + ['V53_74_other_std'] + ['V53_74_other_max'] + ['V53_74_other_pro']

train = data.iloc[:train_length][cols_save]
test  = data.iloc[train_length:][cols_save]

prefix = '526'
dir_save = 'valid_use'

save_feature(train[cols_save], prefix, dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
save_feature(test[cols_save],  prefix, dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)

(590540,) | V53_74_mean
(590540,) | V53_74_sum
(590540,) | V53_74_std
(590540,) | V53_74_max
(590540,) | V53_74_pro
(590540,) | V53_74_other_mean
(590540,) | V53_74_other_sum
(590540,) | V53_74_other_std
(590540,) | V53_74_other_max
(590540,) | V53_74_other_pro
(506691,) | V53_74_mean
(506691,) | V53_74_sum
(506691,) | V53_74_std
(506691,) | V53_74_max
(506691,) | V53_74_pro
(506691,) | V53_74_other_mean
(506691,) | V53_74_other_sum
(506691,) | V53_74_other_std
(506691,) | V53_74_other_max
(506691,) | V53_74_other_pro


In [26]:
#========================================================================
# V75_94
#========================================================================
# display(df_train[cols_V2_11])
# for col in cols_V2_11:
#     display(df_train[col].value_counts(dropna=False).head())
#     display(df_train.groupby(col)[COLUMN_TARGET].mean().head())

if 'V88' in cols_V75_94:
    cols_V75_94.remove('V88')
if 'V89' in cols_V75_94:
    cols_V75_94.remove('V89')
    
cols_other = [f'V{no}' for no in [75, 90, 91, 94, 100, 104, 105, 106]]

print(len(cols_V75_94))
cols_V75_94 = list(set(cols_V75_94) - set(cols_other))
print(len(cols_V75_94))

data['V75_94_mean'] = data[cols_V75_94].mean(axis=1)
data['V75_94_sum']  = data[cols_V75_94].sum(axis=1)
data['V75_94_std']  = data[cols_V75_94].std(axis=1)
data['V75_94_max']  = data[cols_V75_94].max(axis=1)
data['V75_94_pro']  = data[cols_V75_94].product(axis=1)

data['V75_94_other_sum']  = data[cols_other].sum(axis=1)
data['V75_94_other_max']  = data[cols_other].max(axis=1)

cols_save =  ['V75_94_mean'] + ['V75_94_sum'] + ['V75_94_std'] + ['V75_94_max'] + ['V75_94_pro']  \
 + ['V75_94_other_sum'] + ['V75_94_other_max']

train = data.iloc[:train_length][cols_save]
test  = data.iloc[train_length:][cols_save]

prefix = '526'
dir_save = 'valid_use'

save_feature(train[cols_save], prefix, dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
save_feature(test[cols_save],  prefix, dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)

(590540,) | V75_94_mean
(590540,) | V75_94_sum
(590540,) | V75_94_std
(590540,) | V75_94_max
(590540,) | V75_94_pro
(590540,) | V75_94_other_sum
(590540,) | V75_94_other_max
(506691,) | V75_94_mean
(506691,) | V75_94_sum
(506691,) | V75_94_std
(506691,) | V75_94_max
(506691,) | V75_94_pro
(506691,) | V75_94_other_sum
(506691,) | V75_94_other_max


In [36]:
#========================================================================
# V95_137
#========================================================================
# display(df_train[cols_V2_11])
# for col in cols_V2_11:
#     display(df_train[col].value_counts(dropna=False).head())
#     display(df_train.groupby(col)[COLUMN_TARGET].mean().head())

if 'V107' in cols_V95_137:
    cols_V95_137.remove('V107')
cols_V95_137 = list(set(cols_V95_137) - set([f'V{no}' for no in range(130, 138, 1)]))
    
cols_other = [f'V{no}' for no in [96, 98, 99, 100, 104, 105, 106 , 120, 121, 122, 126, 127, 128]]
cols_other_2 = [f'V{no}' for no in [117, 118, 119]]

print(len(cols_V95_137))
cols_V95_137 = sorted(list(set(cols_V95_137) - set(cols_other) -set(cols_other_2)))
print(len(cols_V95_137))

data['V95_137_mean'] = data[cols_V95_137].mean(axis=1)
data['V95_137_sum']  = data[cols_V95_137].sum(axis=1)
data['V95_137_std']  = data[cols_V95_137].std(axis=1)
data['V95_137_max']  = data[cols_V95_137].max(axis=1)
data['V95_137_pro']  = data[cols_V95_137].product(axis=1)

data['V117~119_other_sum']  = data[cols_other_2].sum(axis=1)
data['V117~119_other_max']  = data[cols_other_2].max(axis=1)

cols_save =  ['V95_137_mean'] + ['V95_137_sum'] + ['V95_137_std'] + ['V95_137_max'] + ['V95_137_pro']  \
+ ['V117~119_other_sum'] + ['V117~119_other_max']

train = data.iloc[:train_length][cols_save]
test  = data.iloc[train_length:][cols_save]

prefix = '526'
dir_save = 'valid_use'

save_feature(train[cols_save], prefix, dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
save_feature(test[cols_save],  prefix, dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)

20
17
(590540,) | V95_137_mean
(590540,) | V95_137_sum
(590540,) | V95_137_std
(590540,) | V95_137_max
(590540,) | V95_137_pro
(590540,) | V117~119_other_sum
(590540,) | V117~119_other_max
(506691,) | V95_137_mean
(506691,) | V95_137_sum
(506691,) | V95_137_std
(506691,) | V95_137_max
(506691,) | V95_137_pro
(506691,) | V117~119_other_sum
(506691,) | V117~119_other_max


In [39]:
#========================================================================
# V138_166
#========================================================================
# display(df_train[cols_V2_11])
# for col in cols_V2_11:
#     display(df_train[col].value_counts(dropna=False).head())
#     display(df_train.groupby(col)[COLUMN_TARGET].mean().head())

if 'V107' in cols_V138_166:
    cols_V138_166.remove('V107')
cols_V138_166 = list(set(cols_V138_166) - set([f'V{no}' for no in range(130, 138, 1)]))
    
cols_other = [f'V{no}' for no in range(153, 159, 1)]
no_use_cols = [f'V{no}' for no in range(159, 167, 1)]

print(len(cols_V138_166))
cols_V138_166 = sorted(list(set(cols_V138_166) - set(cols_other) -set(no_use_cols)))
print(len(cols_V138_166))

data['V138_166_mean'] = data[cols_V138_166].mean(axis=1)
data['V138_166_sum']  = data[cols_V138_166].sum(axis=1)
data['V138_166_std']  = data[cols_V138_166].std(axis=1)
data['V138_166_max']  = data[cols_V138_166].max(axis=1)
data['V138_166_pro']  = data[cols_V138_166].product(axis=1)

data['V153~159_sum']  = data[cols_other].sum(axis=1)
data['V153~159_max']  = data[cols_other].max(axis=1)

cols_save =  ['V138_166_mean'] + ['V138_166_sum'] + ['V138_166_std'] + ['V138_166_max'] + ['V138_166_pro']  \
+ ['V153~159_sum'] + ['V153~159_max']

train = data.iloc[:train_length][cols_save]
test  = data.iloc[train_length:][cols_save]

prefix = '526'
dir_save = 'valid_use'

save_feature(train[cols_save], prefix, dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
save_feature(test[cols_save],  prefix, dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)

(590540,) | V138_166_mean
(590540,) | V138_166_sum
(590540,) | V138_166_std
(590540,) | V138_166_max
(590540,) | V138_166_pro
(590540,) | V153~159_sum
(590540,) | V153~159_max
(506691,) | V138_166_mean
(506691,) | V138_166_sum
(506691,) | V138_166_std
(506691,) | V138_166_max
(506691,) | V138_166_pro
(506691,) | V153~159_sum
(506691,) | V153~159_max


In [41]:
#========================================================================
# V167_216
#========================================================================
# display(df_train[cols_V2_11])
# for col in cols_V2_11:
#     display(df_train[col].value_counts(dropna=False).head())
#     display(df_train.groupby(col)[COLUMN_TARGET].mean().head())

cols_V167_216 = list(set(cols_V167_216) - set([f'V{no}' for no in range(130, 138, 1)]))
    
cols_other = [f'V{no}' for no in range(186, 202, 1)]
no_use_cols = [f'V{no}' for no in [169, 172, 173, 174, 175] + list(range(202, 217, 1))]

print(len(cols_V167_216))
cols_V167_216 = sorted(list(set(cols_V167_216) - set(cols_other) -set(no_use_cols)))
print(len(cols_V167_216))

data['V167_216_mean'] = data[cols_V167_216].mean(axis=1)
data['V167_216_sum']  = data[cols_V167_216].sum(axis=1)
data['V167_216_std']  = data[cols_V167_216].std(axis=1)
data['V167_216_max']  = data[cols_V167_216].max(axis=1)
data['V167_216_pro']  = data[cols_V167_216].product(axis=1)

data['V186~201_sum']  = data[cols_other].sum(axis=1)
data['V186~201_max']  = data[cols_other].max(axis=1)

cols_save =  ['V167_216_mean'] + ['V167_216_sum'] + ['V167_216_std'] + ['V167_216_max'] + ['V167_216_pro']  \
+ ['V186~201_sum'] + ['V186~201_max']

train = data.iloc[:train_length][cols_save]
test  = data.iloc[train_length:][cols_save]

prefix = '526'
dir_save = 'valid_use'

save_feature(train[cols_save], prefix, dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
save_feature(test[cols_save],  prefix, dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)

50
14
(590540,) | V167_216_mean
(590540,) | V167_216_sum
(590540,) | V167_216_std
(590540,) | V167_216_max
(590540,) | V167_216_pro
(590540,) | V186~201_sum
(590540,) | V186~201_max
(506691,) | V167_216_mean
(506691,) | V167_216_sum
(506691,) | V167_216_std
(506691,) | V167_216_max
(506691,) | V167_216_pro
(506691,) | V186~201_sum
(506691,) | V186~201_max


In [4]:
#========================================================================
# V216_278
#========================================================================
# display(df_train[cols_V2_11])
# for col in cols_V2_11:
#     display(df_train[col].value_counts(dropna=False).head())
#     display(df_train.groupby(col)[COLUMN_TARGET].mean().head())

cols_V216_278 = list(set(cols_V216_278) - set([f'V{no}' for no in range(130, 138, 1)]))
    
cols_V226_230 = [f'V{no}' for no in [220, 226, 227, 228, 229, 230]]
cols_V242_263 = [f'V{no}' for no in list(range(242, 250, 1)) + list(range(252, 255, 1)) + list(range(257, 263, 1))]
cols_V250_1_255_6 = [f'V{no}' for no in [250,251,255,256]]
no_use_cols = [f'V{no}' for no in [223, 224, 225, 240, 241, 250, 251, 255, 256] + list(range(263, 279, 1))]

print(len(cols_V216_278))
cols_V217_219_231_239 = sorted(list(set(cols_V216_278) 
                                    - set(cols_V226_230) - set(cols_V242_263) -set(no_use_cols)))
print(len(cols_V217_219_231_239))

data['V226_230_mean'] = data[cols_V226_230].mean(axis=1)
data['V226_230_sum']  = data[cols_V226_230].sum(axis=1)
data['V226_230_std']  = data[cols_V226_230].std(axis=1)
data['V226_230_max']  = data[cols_V226_230].max(axis=1)
data['V226_230_pro']  = data[cols_V226_230].product(axis=1)

data['V242_263_mean'] = data[cols_V242_263].mean(axis=1)
data['V242_263_sum']  = data[cols_V242_263].sum(axis=1)
data['V242_263_std']  = data[cols_V242_263].std(axis=1)
data['V242_263_max']  = data[cols_V242_263].max(axis=1)
data['V242_263_pro']  = data[cols_V242_263].product(axis=1)

data['V250_1_255_6_mean'] = data[cols_V250_1_255_6].mean(axis=1)
data['V250_1_255_6_sum']  = data[cols_V250_1_255_6].sum(axis=1)
data['V250_1_255_6_std']  = data[cols_V250_1_255_6].std(axis=1)
data['V250_1_255_6_max']  = data[cols_V250_1_255_6].max(axis=1)
data['V250_1_255_6_pro']  = data[cols_V250_1_255_6].product(axis=1)

data['V217_219_231_239_mean'] = data[cols_V217_219_231_239].mean(axis=1)
data['V217_219_231_239_sum']  = data[cols_V217_219_231_239].sum(axis=1)
data['V217_219_231_239_std']  = data[cols_V217_219_231_239].std(axis=1)
data['V217_219_231_239_max']  = data[cols_V217_219_231_239].max(axis=1)
data['V217_219_231_239_pro']  = data[cols_V217_219_231_239].product(axis=1)

cols_save =  [col for col in data.columns if col.count('V217_219')
              or col.count('V242_263') or col.count('V226_230') or col.count('V250_1_')]

train = data.iloc[:train_length][cols_save]
test  = data.iloc[train_length:][cols_save]
data.drop(cols_save, axis=1, inplace=True)

prefix = '526'
dir_save = 'valid_use'

save_feature(train[cols_save], prefix, dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
save_feature(test[cols_save],  prefix, dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)

SystemExit: 

In [10]:
#========================================================================
# V263~278
#========================================================================
if 0:
    pd.set_option('max_columns', 200)
    pd.set_option('max_rows', 500)
    tmp = data[~data[cols_V263_279[0]].isnull()]
    display(tmp[['TransactionAmt'] + cols_V263_279])
    sys.exit()

col_amt = 'TransactionAmt'

cols_V263_265 = [f'V{no}' for no in range(263, 266, 1)]
cols_V266_268 = [f'V{no}' for no in range(266, 269, 1)]
cols_V270_272 = [f'V{no}' for no in range(270, 273, 1)]
cols_V273_275 = [f'V{no}' for no in range(273, 276, 1)]
cols_V276_278 = [f'V{no}' for no in range(276, 279, 1)]

# V263~265
data['V263_265_std'] = data[cols_V263_265].std(axis=1)
data['V264_div_265'] = data['V264'] / (data['V265'] + 1)
data['V264_div_263'] = data['V264'] / (data['V263'] + 1)
data['V265_div_263'] = data['V265'] / (data['V263'] + 1)
data['V263_div_Amt'] = data['V263'] / (data[col_amt] + 1)
data['V264_div_Amt'] = data['V264'] / (data[col_amt] + 1)
data['V265_div_Amt'] = data['V265'] / (data[col_amt] + 1)

# V263~265とV266~268
data['V263_div_266'] = data['V263'] / (data['V266'] + 1)
data['V263_div_267'] = data['V263'] / (data['V267'] + 1)
data['V263_div_268'] = data['V263'] / (data['V268'] + 1)
data['V264_div_266'] = data['V264'] / (data['V266'] + 1)
data['V264_div_267'] = data['V264'] / (data['V267'] + 1)
data['V264_div_266'] = data['V264'] / (data['V266'] + 1)
data['V265_div_267'] = data['V265'] / (data['V267'] + 1)
data['V265_div_268'] = data['V265'] / (data['V268'] + 1)
data['V265_div_268'] = data['V265'] / (data['V268'] + 1)

# V263~265とV270~272
data['V263_div_270'] = data['V263'] / (data['V270'] + 1)
data['V263_div_271'] = data['V263'] / (data['V271'] + 1)
data['V263_div_272'] = data['V263'] / (data['V272'] + 1)
data['V264_div_270'] = data['V264'] / (data['V270'] + 1)
data['V264_div_271'] = data['V264'] / (data['V271'] + 1)
data['V264_div_272'] = data['V264'] / (data['V272'] + 1)
data['V265_div_270'] = data['V265'] / (data['V270'] + 1)
data['V265_div_271'] = data['V265'] / (data['V271'] + 1)
data['V265_div_272'] = data['V265'] / (data['V272'] + 1)

# V263~265とV273~275
data['V263_div_273'] = data['V263'] / (data['V273'] + 1)
data['V263_div_274'] = data['V263'] / (data['V274'] + 1)
data['V263_div_275'] = data['V263'] / (data['V275'] + 1)
data['V264_div_273'] = data['V264'] / (data['V273'] + 1)
data['V264_div_274'] = data['V264'] / (data['V274'] + 1)
data['V264_div_275'] = data['V264'] / (data['V275'] + 1)
data['V265_div_273'] = data['V265'] / (data['V273'] + 1)
data['V265_div_274'] = data['V265'] / (data['V274'] + 1)
data['V265_div_275'] = data['V265'] / (data['V275'] + 1)

# V263~265とV276~278
data['V263_div_276'] = data['V263'] / (data['V276'] + 1)
data['V263_div_277'] = data['V263'] / (data['V277'] + 1)
data['V263_div_278'] = data['V263'] / (data['V278'] + 1)
data['V264_div_276'] = data['V264'] / (data['V276'] + 1)
data['V264_div_277'] = data['V264'] / (data['V277'] + 1)
data['V264_div_278'] = data['V264'] / (data['V278'] + 1)
data['V265_div_276'] = data['V265'] / (data['V276'] + 1)
data['V265_div_277'] = data['V265'] / (data['V277'] + 1)
data['V265_div_278'] = data['V265'] / (data['V278'] + 1)


# V266~268
data['V266_268_std'] = data[cols_V266_268].std(axis=1)
data['V267_div_268'] = data['V267'] / (data['V268'] + 1)
data['V267_div_266'] = data['V267'] / (data['V266'] + 1)
data['V268_div_266'] = data['V268'] / (data['V266'] + 1)
data['V266_div_Amt'] = data['V266'] / (data[col_amt] + 1)
data['V267_div_Amt'] = data['V267'] / (data[col_amt] + 1)
data['V268_div_Amt'] = data['V268'] / (data[col_amt] + 1)

# V266~268とV270~272
data['V266_div_270'] = data['V266'] / (data['V270'] + 1)
data['V266_div_271'] = data['V266'] / (data['V271'] + 1)
data['V266_div_272'] = data['V266'] / (data['V272'] + 1)
data['V267_div_270'] = data['V267'] / (data['V270'] + 1)
data['V267_div_271'] = data['V267'] / (data['V271'] + 1)
data['V267_div_272'] = data['V267'] / (data['V272'] + 1)
data['V268_div_270'] = data['V268'] / (data['V270'] + 1)
data['V268_div_271'] = data['V268'] / (data['V271'] + 1)
data['V268_div_272'] = data['V268'] / (data['V272'] + 1)

# V266~268とV273~275
data['V266_div_273'] = data['V266'] / (data['V273'] + 1)
data['V266_div_274'] = data['V266'] / (data['V274'] + 1)
data['V266_div_275'] = data['V266'] / (data['V275'] + 1)
data['V267_div_273'] = data['V267'] / (data['V273'] + 1)
data['V267_div_274'] = data['V267'] / (data['V274'] + 1)
data['V267_div_275'] = data['V267'] / (data['V275'] + 1)
data['V268_div_273'] = data['V268'] / (data['V273'] + 1)
data['V268_div_274'] = data['V268'] / (data['V274'] + 1)
data['V268_div_275'] = data['V268'] / (data['V275'] + 1)

# V266~268とV276~278
data['V266_div_276'] = data['V266'] / (data['V276'] + 1)
data['V266_div_277'] = data['V266'] / (data['V277'] + 1)
data['V266_div_278'] = data['V266'] / (data['V278'] + 1)
data['V267_div_276'] = data['V267'] / (data['V276'] + 1)
data['V267_div_277'] = data['V267'] / (data['V277'] + 1)
data['V267_div_278'] = data['V267'] / (data['V278'] + 1)
data['V268_div_276'] = data['V268'] / (data['V276'] + 1)
data['V268_div_277'] = data['V268'] / (data['V277'] + 1)
data['V268_div_278'] = data['V268'] / (data['V278'] + 1)


# V270~272
data['V270_272_std'] = data[cols_V270_272].std(axis=1)
data['V271_div_272'] = data['V271'] / (data['V272'] + 1)
data['V271_div_270'] = data['V271'] / (data['V270'] + 1)
data['V272_div_270'] = data['V272'] / (data['V270'] + 1)
data['V270_div_Amt'] = data['V270'] / (data[col_amt] + 1)
data['V271_div_Amt'] = data['V271'] / (data[col_amt] + 1)
data['V272_div_Amt'] = data['V272'] / (data[col_amt] + 1)

# V270~272とV273~275
data['V270_div_273'] = data['V270'] / (data['V273'] + 1)
data['V270_div_274'] = data['V270'] / (data['V274'] + 1)
data['V270_div_275'] = data['V270'] / (data['V275'] + 1)
data['V271_div_273'] = data['V271'] / (data['V273'] + 1)
data['V271_div_274'] = data['V271'] / (data['V274'] + 1)
data['V271_div_275'] = data['V271'] / (data['V275'] + 1)
data['V272_div_273'] = data['V272'] / (data['V273'] + 1)
data['V272_div_274'] = data['V272'] / (data['V274'] + 1)
data['V272_div_275'] = data['V272'] / (data['V275'] + 1)

# V270~272とV276~278
data['V270_div_276'] = data['V270'] / (data['V276'] + 1)
data['V270_div_277'] = data['V270'] / (data['V277'] + 1)
data['V270_div_278'] = data['V270'] / (data['V278'] + 1)
data['V271_div_276'] = data['V271'] / (data['V276'] + 1)
data['V271_div_277'] = data['V271'] / (data['V277'] + 1)
data['V271_div_278'] = data['V271'] / (data['V278'] + 1)
data['V272_div_276'] = data['V272'] / (data['V276'] + 1)
data['V272_div_277'] = data['V272'] / (data['V277'] + 1)
data['V272_div_278'] = data['V272'] / (data['V278'] + 1)


# V273~275
data['V273_275_std'] = data[cols_V273_275].std(axis=1)
data['V274_div_275'] = data['V274'] / (data['V275'] + 1)
data['V274_div_273'] = data['V274'] / (data['V273'] + 1)
data['V275_div_273'] = data['V275'] / (data['V273'] + 1)
data['V273_div_Amt'] = data['V273'] / (data[col_amt] + 1)
data['V274_div_Amt'] = data['V274'] / (data[col_amt] + 1)
data['V275_div_Amt'] = data['V275'] / (data[col_amt] + 1)

# V273~275とV276~278
data['V273_div_276'] = data['V273'] / (data['V276'] + 1)
data['V273_div_277'] = data['V273'] / (data['V277'] + 1)
data['V273_div_278'] = data['V273'] / (data['V278'] + 1)
data['V274_div_276'] = data['V274'] / (data['V276'] + 1)
data['V274_div_277'] = data['V274'] / (data['V277'] + 1)
data['V274_div_278'] = data['V274'] / (data['V278'] + 1)
data['V275_div_276'] = data['V275'] / (data['V276'] + 1)
data['V275_div_277'] = data['V275'] / (data['V277'] + 1)
data['V275_div_278'] = data['V275'] / (data['V278'] + 1)

# V276~278
data['V276_278_std'] = data[cols_V276_278].std(axis=1)
data['V277_div_278'] = data['V277'] / (data['V278'] + 1)
data['V277_div_276'] = data['V277'] / (data['V276'] + 1)
data['V278_div_276'] = data['V278'] / (data['V276'] + 1)
data['V276_div_Amt'] = data['V276'] / (data[col_amt] + 1)
data['V277_div_Amt'] = data['V277'] / (data[col_amt] + 1)
data['V278_div_Amt'] = data['V278'] / (data[col_amt] + 1)


cols_save =  [col for col in data.columns if col.count('_div_') or col.count('_std')]

train = data.iloc[:train_length][cols_save]
test  = data.iloc[train_length:][cols_save]
data.drop(cols_save, axis=1, inplace=True)

prefix = '526'
dir_save = 'valid_use'

save_feature(train[cols_save], prefix, dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
save_feature(test[cols_save],  prefix, dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)

(506691,) | V265_div_274
(506691,) | V265_div_275
(506691,) | V263_div_276
(506691,) | V263_div_277
(506691,) | V263_div_278
(506691,) | V264_div_276
(506691,) | V264_div_277
(506691,) | V264_div_278
(506691,) | V265_div_276
(506691,) | V265_div_277
(506691,) | V265_div_278
(506691,) | V266_268_std
(506691,) | V267_div_268
(506691,) | V267_div_266
(506691,) | V268_div_266
(506691,) | V266_div_Amt
(506691,) | V267_div_Amt
(506691,) | V268_div_Amt
(506691,) | V266_div_270
(506691,) | V266_div_271
(506691,) | V266_div_272
(506691,) | V267_div_270
(506691,) | V267_div_271
(506691,) | V267_div_272
(506691,) | V268_div_270
(506691,) | V268_div_271
(506691,) | V268_div_272
(506691,) | V266_div_273
(506691,) | V266_div_274
(506691,) | V266_div_275
(506691,) | V267_div_273
(506691,) | V267_div_274
(506691,) | V267_div_275
(506691,) | V268_div_273
(506691,) | V268_div_274
(506691,) | V268_div_275
(506691,) | V266_div_276
(506691,) | V266_div_277
(506691,) | V266_div_278
(506691,) | V267_div_276


In [ ]:
#========================================================================
# V279_321
#========================================================================
# display(df_train[cols_V2_11])
# for col in cols_V2_11:
#     display(df_train[col].value_counts(dropna=False).head())
#     display(df_train.groupby(col)[COLUMN_TARGET].mean().head())

# good
cols_V279_284 = [f'V{no}' for no in list(range(285, 284, 1))]
cols_V289_295 = [f'V{no}' for no in list(range(289, 295, 1))]
cols_V302_304 = [f'V{no}' for no in list(range(302, 305, 1))]

# not bad
cols_V285_288 = [f'V{no}' for no in list(range(285, 289, 1))]
cols_V296_301 = [f'V{no}' for no in list(range(296, 302, 1))]
cols_V306_321 = [f'V{no}' for no in list(range(306, 322, 1))]

data['V279_284_mean'] = data[cols_V279_284].mean(axis=1)
data['V279_284_sum']  = data[cols_V279_284].sum(axis=1)
data['V279_284_std']  = data[cols_V279_284].std(axis=1)
data['V279_284_max']  = data[cols_V279_284].max(axis=1)
data['V279_284_pro']  = data[cols_V279_284].product(axis=1)

data['V289_295_mean'] = data[cols_V289_295].mean(axis=1)
data['V289_295_sum']  = data[cols_V289_295].sum(axis=1)
data['V289_295_std']  = data[cols_V289_295].std(axis=1)
data['V289_295_max']  = data[cols_V289_295].max(axis=1)
data['V289_295_pro']  = data[cols_V289_295].product(axis=1)

data['V302_304_mean'] = data[cols_V302_304].mean(axis=1)
data['V302_304_sum']  = data[cols_V302_304].sum(axis=1)
data['V302_304_std']  = data[cols_V302_304].std(axis=1)
data['V302_304_max']  = data[cols_V302_304].max(axis=1)
data['V302_304_pro']  = data[cols_V302_304].product(axis=1)

data['V285_288_mean'] = data[cols_V285_288].mean(axis=1)
data['V285_288_sum']  = data[cols_V285_288].sum(axis=1)
data['V285_288_std']  = data[cols_V285_288].std(axis=1)
data['V285_288_max']  = data[cols_V285_288].max(axis=1)
data['V285_288_pro']  = data[cols_V285_288].product(axis=1)

data['V296_301_mean'] = data[cols_V296_301].mean(axis=1)
data['V296_301_sum']  = data[cols_V296_301].sum(axis=1)
data['V296_301_std']  = data[cols_V296_301].std(axis=1)
data['V296_301_max']  = data[cols_V296_301].max(axis=1)
data['V296_301_pro']  = data[cols_V296_301].product(axis=1)

data['V306_321_mean'] = data[cols_V306_321].mean(axis=1)
data['V306_321_sum']  = data[cols_V306_321].sum(axis=1)
data['V306_321_std']  = data[cols_V306_321].std(axis=1)
data['V306_321_max']  = data[cols_V306_321].max(axis=1)
data['V306_321_pro']  = data[cols_V306_321].product(axis=1)

cols_save =  [col for col in data.columns if col.count('V279_284')
              or col.count('V289_295') or col.count('V302_304') or col.count('V285_288')
              or col.count('V296_301') or col.count('V306_321')
             ]

train = data.iloc[:train_length][cols_save]
test  = data.iloc[train_length:][cols_save]
data.drop(cols_save, axis=1, inplace=True)

prefix = '526'
dir_save = 'valid_use'

save_feature(train[cols_save], prefix, dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
save_feature(test[cols_save],  prefix, dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)

In [16]:
for col in sorted(cols_V279_321):
# for col in sorted(cols_V242_263):
    tmp = df_train.groupby(col)[COLUMN_TARGET].agg({
        'mean': 'mean',
        'count': 'count',
    })
    display(tmp.head(10))

,mean,count
V279,,
0.0,0.028626,492169
1.0,0.059149,60474
2.0,0.072825,18826
3.0,0.085454,8016
4.0,0.089826,4030
5.0,0.095102,2103
6.0,0.095547,1235
7.0,0.110054,736
8.0,0.106904,449


,mean,count
V280,,
0.0,0.027337,422464
1.0,0.040778,90882
2.0,0.052997,32870
3.0,0.067477,15309
4.0,0.078297,8340
5.0,0.085057,5173
6.0,0.092935,3411
7.0,0.098537,2324
8.0,0.110630,1618


,mean,count
V281,,
0.0,0.032149,561201
1.0,0.072917,16896
2.0,0.101862,5694
3.0,0.106683,2484
4.0,0.187883,1469
5.0,0.132450,604
6.0,0.076923,351
7.0,0.222222,225
8.0,0.039216,102


,mean,count
V282,,
0.0,0.016714,212812
1.0,0.037983,316455
2.0,0.074220,37537
3.0,0.088132,12243
4.0,0.097468,4976
5.0,0.125545,2525
6.0,0.141593,1130
7.0,0.090491,652
8.0,0.073718,312


,mean,count
V283,,
0.0,0.016648,211499
1.0,0.035601,284599
2.0,0.057907,51773
3.0,0.072061,18984
4.0,0.090664,8912
5.0,0.112614,4955
6.0,0.106077,2847
7.0,0.104651,1720
8.0,0.132379,1118


...

,mean,count
V317,,
0.0000,0.027015,519413
0.2915,1.000000,1
0.4669,1.000000,1
0.5830,1.000000,1
0.6042,1.000000,1
0.6413,1.000000,1
0.8782,0.000000,1
0.9286,0.000000,2
0.9328,1.000000,1


,mean,count
V318,,
0.0000,0.028159,535074
0.2915,1.000000,1
0.4669,1.000000,2
0.5830,1.000000,1
0.6413,1.000000,1
0.8782,0.000000,3
0.9286,0.000000,2
0.9328,1.000000,1
0.9338,1.000000,1


,mean,count
V319,,
0.0000,0.03443,562740
1.9939,0.00000,1
2.1518,0.00000,1
2.2578,0.00000,1
2.2960,0.00000,1
2.3362,0.00000,1
2.5334,0.00000,1
2.5822,0.00000,1
2.9680,0.00000,1


,mean,count
V320,,
0.0000,0.033657,542892
1.3086,0.000000,6
1.9939,0.000000,1
2.2578,0.000000,1
2.2917,0.000000,1
2.2960,0.000000,1
2.3362,0.000000,1
2.5334,0.000000,1
2.5822,0.000000,1


,mean,count
V321,,
0.0000,0.033904,555629
1.3086,0.000000,6
1.9939,0.000000,1
2.1518,0.000000,1
2.2578,0.000000,1
2.2960,0.000000,1
2.3362,0.000000,1
2.5334,0.000000,1
2.5822,0.000000,1


In [19]:
# for col in sorted(no_use_cols):
list_pro = data['ProductCD'].unique()
# for col in sorted(cols_V242_263):
for col in sorted(cols_V279_321):
    list_df = []
    for pro in list_pro:
        tmp = data[data['ProductCD']==pro]
        tmp = tmp.groupby(col)[COLUMN_TARGET].agg({
            f'{pro}_mean' : 'mean',
            f'{pro}_count': 'count',
        })
        list_df.append(tmp)
    df = pd.concat(list_df, axis=1)
    df.sort_values(by='C_count', inplace=True, ascending=False)
    print(col)
    display(df.head(10))

V279


,W_mean,W_count,H_mean,H_count,C_mean,C_count,S_mean,S_count,R_mean,R_count
V279,,,,,,,,,,
0.0,0.013750,370683.0,0.042074,27903.0,0.118378,53667.0,0.047776,8184,0.033846,31732.0
1.0,0.042214,44677.0,0.076568,3252.0,0.136759,7824.0,0.117030,1239,0.065192,3482.0
2.0,0.066979,13034.0,0.076784,1107.0,0.094076,3072.0,0.151961,408,0.051452,1205.0
3.0,0.086060,5287.0,0.086681,473.0,0.081978,1537.0,0.188889,180,0.053803,539.0
4.0,0.097552,2614.0,0.090909,198.0,0.070406,838.0,0.180000,100,0.042857,280.0
5.0,0.108009,1361.0,0.092593,54.0,0.061441,472.0,0.206897,58,0.044304,158.0
6.0,0.114733,767.0,0.047619,21.0,0.043189,301.0,0.234043,47,0.050505,99.0
7.0,0.141280,453.0,0.100000,10.0,0.038251,183.0,0.250000,28,0.032258,62.0
8.0,0.147170,265.0,0.000000,3.0,0.024194,124.0,0.222222,18,0.051282,39.0


V280


,W_mean,W_count,H_mean,H_count,C_mean,C_count,S_mean,S_count,R_mean,R_count
V280,,,,,,,,,,
0.0,0.010944,307281.0,0.040684,27382.0,0.115466,50084.0,0.037222,6394,0.033554,31323.0
1.0,0.024262,72911.0,0.079300,3430.0,0.136792,9445.0,0.094538,1428,0.064885,3668.0
2.0,0.041389,25804.0,0.082458,1237.0,0.108667,3773.0,0.117722,790,0.054502,1266.0
3.0,0.058589,11777.0,0.088235,544.0,0.101904,1943.0,0.144086,465,0.051724,580.0
4.0,0.072557,6395.0,0.099206,252.0,0.089401,1085.0,0.181507,292,0.044304,316.0
5.0,0.081858,4068.0,0.073684,95.0,0.085443,632.0,0.187817,197,0.049724,181.0
6.0,0.095875,2691.0,0.076923,52.0,0.075980,408.0,0.133803,142,0.042373,118.0
7.0,0.103723,1880.0,0.111111,18.0,0.072289,249.0,0.114286,105,0.027778,72.0
8.0,0.118008,1305.0,0.000000,8.0,0.076023,171.0,0.119048,84,0.040000,50.0


V281


...

,W_mean,W_count,H_mean,H_count,C_mean,C_count,S_mean,S_count,R_mean,R_count
V319,,,,,,,,,,
0.000000,0.019687,421961.0,0.045225,31067.0,0.118743,65831.0,0.061124,9538.0,0.036776,34343.0
16.790400,NaN,NaN,NaN,NaN,0.071429,14.0,NaN,NaN,NaN,NaN
22.053301,NaN,NaN,NaN,NaN,0.000000,13.0,NaN,NaN,NaN,NaN
24.837900,NaN,NaN,NaN,NaN,0.000000,12.0,NaN,NaN,NaN,NaN
24.651400,NaN,NaN,NaN,NaN,0.000000,12.0,NaN,NaN,NaN,NaN
44.106602,NaN,NaN,NaN,NaN,0.166667,12.0,NaN,NaN,NaN,NaN
14.808200,NaN,NaN,NaN,NaN,0.000000,11.0,NaN,NaN,NaN,NaN
29.264500,NaN,NaN,NaN,NaN,0.000000,11.0,NaN,NaN,NaN,NaN
42.558998,NaN,NaN,NaN,NaN,0.000000,11.0,NaN,NaN,NaN,NaN


V320


,W_mean,W_count,H_mean,H_count,C_mean,C_count,S_mean,S_count,R_mean,R_count
V320,,,,,,,,,,
0.000000,0.018596,404334.0,0.044516,30888.0,0.117732,64630.0,0.058757,8867.0,0.03652,34173.0
4.108600,NaN,NaN,NaN,NaN,0.075758,66.0,NaN,NaN,NaN,NaN
8.808600,NaN,NaN,NaN,NaN,0.000000,28.0,NaN,NaN,NaN,NaN
12.826000,NaN,NaN,NaN,NaN,0.166667,24.0,NaN,NaN,NaN,NaN
96.671997,NaN,NaN,NaN,NaN,0.956522,23.0,NaN,NaN,NaN,NaN
26.118401,NaN,NaN,NaN,NaN,0.450000,20.0,NaN,NaN,NaN,NaN
200.178802,NaN,NaN,NaN,NaN,0.000000,19.0,NaN,NaN,NaN,NaN
58.003201,NaN,NaN,NaN,NaN,1.000000,18.0,NaN,NaN,NaN,NaN
35.345699,NaN,NaN,NaN,NaN,0.294118,17.0,NaN,NaN,NaN,NaN


V321


,W_mean,W_count,H_mean,H_count,C_mean,C_count,S_mean,S_count,R_mean,R_count
V321,,,,,,,,,,
0.000000,0.019006,415865.0,0.044633,30941.0,0.118269,65351.0,0.061357,9241.0,0.036721,34231.0
150.011093,NaN,NaN,NaN,NaN,0.000000,19.0,NaN,NaN,NaN,NaN
13.292400,NaN,NaN,NaN,NaN,0.000000,18.0,NaN,NaN,NaN,NaN
4.108600,NaN,NaN,NaN,NaN,0.055556,18.0,NaN,NaN,NaN,NaN
158.342804,NaN,NaN,NaN,NaN,1.000000,17.0,NaN,NaN,NaN,NaN
16.790400,NaN,NaN,NaN,NaN,0.071429,14.0,NaN,NaN,NaN,NaN
22.053301,NaN,NaN,NaN,NaN,0.000000,13.0,NaN,NaN,NaN,NaN
24.837900,NaN,NaN,NaN,NaN,0.000000,13.0,NaN,NaN,NaN,NaN
44.106602,NaN,NaN,NaN,NaN,0.166667,12.0,NaN,NaN,NaN,NaN


In [20]:
list_pro

'R'